In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
file_path = 'Car_Prediction_Splitdata/big/scaled_big_car_used_data_without_outliers.csv'
file_name_with_extension = os.path.basename(file_path)
file_name, file_extension = os.path.splitext(file_name_with_extension)

In [3]:
import csv
import math

data = [
    ['RandomForestRegressor', None, None, None, None, None, None],
    ['DecisionTreeRegressor', None, None, None, None, None, None],
    ['XGBRegressor', None, None, None, None, None, None],
    ['BaggingRegressor', None, None, None, None, None, None]
]

# Tên tiêu đề cột
header = ['Regressor', 'R2_Score','Cross validation score', 'mean_squared_error', 'mean_absolute_error', 'root_mean_squared_error', 'Diff']

file_name = file_name + '_result.csv'

# Tạo và ghi vào file CSV
with open(file_name, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(data)

In [4]:
def write_result(name,r2_score_value,scores_value,mean_squared_error,mean_absolute_error,root_mean_squared_error,diff_value):
    data = []
    with open(file_name, 'r') as file:
        reader = csv.reader(file)
        data = list(reader)
    for row in data:
        if row[0] == name:
            row[1] = r2_score_value
            row[2] = scores_value
            row[3] = mean_squared_error
            row[4] = mean_absolute_error
            row[5] = root_mean_squared_error
            row[6] = diff_value

    # Ghi lại dữ liệu đã cập nhật vào file CSV
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

In [5]:
df= pd.read_csv('Car_Prediction_Splitdata/big/scaled_big_car_used_data_without_outliers.csv')
df.head()

,maker_model,variants,body_type,type,seat,door,mileage,non_smoker_vehicle,power,gearbox,engine_size,fuel_type,emission_classe,colour,manufacturer_colour,upholstery_colour,upholstery,car_age,inspection_interval,price (€)
0,-1.500352,0.218739,0.433270,0.419918,0.155740,-0.434859,0.002173,-1.731720,1.184470,-0.622728,1.659787,-0.893815,0.676819,-0.013562,-0.329873,1.475588,-0.112520,-0.767191,0.466211,72239
1,1.379300,1.072728,1.892357,0.419918,0.155740,0.746877,1.234424,-1.731720,0.001517,-0.622728,0.167679,-1.030832,-0.730657,0.214446,-0.336681,-0.341443,-0.818773,0.363325,0.466211,22970
2,1.140293,0.625774,1.892357,0.419918,-2.087474,-0.434859,0.080873,0.577461,-0.843449,-0.622728,0.167679,-1.030832,-0.730657,1.126480,-0.916674,-0.341443,1.299987,-0.390352,-0.960946,27999
3,-1.249780,1.603253,0.676451,0.419918,0.155740,-0.434859,0.453086,0.577461,1.860443,-0.622728,1.659787,-0.071712,-0.730657,-1.153605,1.122832,-1.249958,-0.112520,0.363325,0.466211,38499
4,-0.004629,1.528637,0.676451,0.419918,0.155740,0.746877,-0.318531,-1.731720,1.606954,-0.622728,0.211677,-0.208729,0.676819,0.898472,0.299133,-0.341443,1.299987,-0.390352,0.466211,38560


In [6]:
X = df.drop(['price (€)'], axis = 1)
Y = df['price (€)']

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import  GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error,  r2_score
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=.3)
print('Training feature matrix size:',X_train.shape)
print('Training target vector size:',Y_train.shape)
print('Test feature matrix size:',X_test.shape)
print('Test target vector size:',Y_test.shape)

Training feature matrix size: (7319, 19)
Training target vector size: (7319,)
Test feature matrix size: (3138, 19)
Test target vector size: (3138,)


In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
RFR=RandomForestRegressor()
RFR.fit(X_train,Y_train)
pred=RFR.predict(X_test)
R2_score = r2_score(Y_test,pred)*100
print('R2_score:',R2_score)
print('mean_squared_error:',mean_squared_error(Y_test,pred))
print('mean_absolute_error:',mean_absolute_error(Y_test,pred))
print('root_mean_squared_error:',np.sqrt(mean_squared_error(Y_test,pred)))

# Cross Validation Score
scores = cross_val_score(RFR, X, Y, cv = 10).mean()*100
print("\nCross validation score :", scores)

# Difference between Accuracy and CV Score
diff = R2_score - scores
print("\nR2_Score - Cross Validation Score :", diff)

write_result('RandomForestRegressor',R2_score,scores,mean_squared_error(Y_test,pred),mean_absolute_error(Y_test,pred),np.sqrt(mean_squared_error(Y_test,pred)),diff)

R2_score: 90.17223427603717
mean_squared_error: 29122820.08798912
mean_absolute_error: 3491.3686917635337
root_mean_squared_error: 5396.556317503702

Cross validation score : 91.15182100525054

R2_Score - Cross Validation Score : -0.9795867292133664


In [12]:
XGB=XGBRegressor()
XGB.fit(X_train,Y_train)
pred=XGB.predict(X_test)
R2_score = r2_score(Y_test,pred)*100
print('R2_score:',R2_score)
print('mean_squared_error:',mean_squared_error(Y_test,pred))
print('mean_absolute_error:',mean_absolute_error(Y_test,pred))
print('root_mean_squared_error:',np.sqrt(mean_squared_error(Y_test,pred)))

# Cross Validation on XGB Model
scores = cross_val_score(XGB, X, Y, cv = 10).mean()*100
print("\nCross validation score :", scores)

# Difference between Accuracy and CV Score
diff = R2_score - scores
print("\nR2_Score - Cross Validation Score :", diff)

write_result('XGBRegressor',R2_score,scores,mean_squared_error(Y_test,pred),mean_absolute_error(Y_test,pred),np.sqrt(mean_squared_error(Y_test,pred)),diff)

R2_score: 92.02773619362792
mean_squared_error: 23624373.1330362
mean_absolute_error: 3176.498517767649
root_mean_squared_error: 4860.49103826313

Cross validation score : 92.77668391962773

R2_Score - Cross Validation Score : -0.7489477259998125


In [13]:
DTR=DecisionTreeRegressor()
DTR.fit(X_train,Y_train)
pred=DTR.predict(X_test)
R2_score = r2_score(Y_test,pred)*100
print('R2_score:',R2_score)
print('mean_squared_error:',mean_squared_error(Y_test,pred))
print('mean_absolute_error:',mean_absolute_error(Y_test,pred))
print('root_mean_squared_error:',np.sqrt(mean_squared_error(Y_test,pred)))
# Cross Validation Score
scores = cross_val_score(DTR, X, Y, cv = 10).mean()*100
print("\nCross validation score :", scores)

# Difference between Accuracy and CV Score
diff = R2_score - scores
print("\nR2_Score - Cross Validation Score :", diff)

write_result('DecisionTreeRegressor',R2_score,scores,mean_squared_error(Y_test,pred),mean_absolute_error(Y_test,pred),np.sqrt(mean_squared_error(Y_test,pred)),diff)

R2_score: 79.59357710066446
mean_squared_error: 60470772.24151972
mean_absolute_error: 4650.18854896962
root_mean_squared_error: 7776.295534605132

Cross validation score : 80.8095749355916

R2_Score - Cross Validation Score : -1.215997834927137


In [14]:
BR=BaggingRegressor()
BR.fit(X_train,Y_train)
pred=BR.predict(X_test)
R2_score = r2_score(Y_test,pred)*100
print('R2_score:',R2_score)
print('mean_squared_error:',mean_squared_error(Y_test,pred))
print('mean_absolute_error:',mean_absolute_error(Y_test,pred))
print('root_mean_squared_error:',np.sqrt(mean_squared_error(Y_test,pred)))

# Cross Validation Score
scores = cross_val_score(BR, X, Y, cv = 10).mean()*100
print("\nCross validation score :", scores)

# Difference between Accuracy and CV Score
diff = R2_score - scores
print("\nR2_Score - Cross Validation Score :", diff)

write_result('BaggingRegressor',R2_score,scores,mean_squared_error(Y_test,pred),mean_absolute_error(Y_test,pred),np.sqrt(mean_squared_error(Y_test,pred)),diff)

R2_score: 89.0274923926791
mean_squared_error: 32515057.230449222
mean_absolute_error: 3673.5893983884184
root_mean_squared_error: 5702.197579043471

Cross validation score : 90.06772598426747

R2_Score - Cross Validation Score : -1.0402335915883754


In [15]:
from sklearn.model_selection import GridSearchCV